In [26]:
import formulallm.formula as f
from autogen import ConversableAgent

In [27]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [13]:
dsl_path = input("Input path to 4ml file")
mapping_condition = input("Input mapping condition")

Input path to 4ml file ./data/MappingExample.4ml
Input mapping condition Mapping.conforms


In [41]:
import io
import contextlib

stdout_buffer = io.StringIO()
stderr_buffer = io.StringIO()

with contextlib.redirect_stdout(stdout_buffer), contextlib.redirect_stderr(stderr_buffer):
    try:
        code = f.load(dsl_path)
        f.solve("pm","1",mapping_condition)
        f.extract("0","0","0")
    except Exception as e:
        pass

message = stdout_buffer.getvalue()
error_message = stderr_buffer.getvalue()

print("Captured stdout:", message)
print("Captured stderr:", error_message)

Captured stdout: line 8:0 mismatched input 'print' expecting {<EOF>, 'domain', 'model', 'transform', 'machine', 'partial', '['}
Error at line 8 column 0: found token print. Expected: 
EOF, DOMAIN, MODEL, TRANSFORM, MACHINE, PARTIAL, LSBRACE, (Failed) MappingExample.4ml
MappingExample.4ml (0, 0): File access error - Syntax error in 4ml file.
MappingExample.4ml (0, 0): The install operation failed
0.01s.
Parsing text took: 0
Visiting text took: 0
No module with that name
0.00s.
Model not solvable. Unsat core terms below.
Conflicts: Mapping.badMapping 
Conflicts: Mapping.invalidUtilization 

0.00s.

Captured stderr: 


In [42]:
agent = ConversableAgent(
    name = "fixer",
    llm_config = local_llm_config,
    human_input_mode="NEVER",
    system_message = '''You are a code fixer. Given a DSL domain and paritial model pair written in formula, which is potentially unsolvable, modify the provided code to make it solvable. 
when suggesting repairs, prioritize modifying the partial model over modifying the domain constraints. You will also be provided with messages that might explain why the DSL is unsolvable.
Do not say anything said isn't code, just provide the entire modified code in formula. Make sure you follow the syntax. For example, 

WorkFlow:
1.understand the domain and the constraints.
2.using the constraints from the domain, modify the partial model. Be as abstract as possible. i.e. use variables like x and y instead of fixed numbers
3.reply with MODIFIED CODE ONLY. Do not speak a single word that is not part of the code. if you do so, i will destroy you
'''
)

In [43]:
reply = agent.generate_reply(messages = [{"content": f"Here is the code: {code} \nHere is the messages: {message}", "role": "user"} ])

In [44]:
print(reply)

domain Mapping
{
  Component ::= new (id: Integer, utilization: Real).
  Processor ::= new (id: Integer).
  Mapping   ::= new (c: Component, p: Processor).

  // The utilization must be > 0
  invalidUtilization :- c is Component, c.utilization <= 0.

  badMapping :- p is Processor, 
    s = sum(0.0, { c.utilization |
                  c is Component, Mapping(c, p) }), s > 100.
  
  conforms no badMapping, no invalidUtilization.
}

partial model pm of Mapping
{
  x = 10.0.
  y = 90.0.
  c1 is Component(0, x).
  c2 is Component(1, y).
  p1 is Processor(0).
  c1.utilization + c2.utilization <= 100.
  Mapping(c1, p1).
}
